In [20]:
import pandas as pd
import numpy as np
from random import shuffle
import os
import scipy
import sqlite3
from collections import defaultdict

In [2]:
curr_path = os.path.realpath('')
print(curr_path)
conn = sqlite3.connect(curr_path+"/lending-club-loan-data/database.sqlite")
cur = conn.cursor()
cur.execute("select * from loan limit 5;")
results = cur.fetchall()
# print(results)
cur.close()
conn.close()

C:\Users\andre\Documents\GitHub\loan_driving2


In [3]:
conn = sqlite3.connect("lending-club-loan-data/database.sqlite")
df = pd.read_sql_query("select loan_amnt,id,loan_status,sub_grade from loan where loan_status = 'Charged Off';", conn)
print(df)
conn.close()

       loan_amnt        id  loan_status sub_grade
0         2500.0   1077430  Charged Off        C4
1         5600.0   1071795  Charged Off        F2
2         5375.0   1071570  Charged Off        B5
3         9000.0   1064687  Charged Off        C1
4        10000.0   1069057  Charged Off        B2
5        21000.0   1039153  Charged Off        B4
6         6000.0   1069559  Charged Off        B3
7        15000.0   1069800  Charged Off        C2
8         5000.0   1069657  Charged Off        D2
9         5000.0   1069465  Charged Off        A5
10       15000.0   1069248  Charged Off        B1
11       12000.0   1069243  Charged Off        C5
12       21000.0   1069410  Charged Off        E4
13       10000.0   1069126  Charged Off        B2
14       12500.0   1069093  Charged Off        D3
15        8200.0   1068906  Charged Off        F2
16        6400.0   1060981  Charged Off        D2
17       17500.0   1068542  Charged Off        D3
18        6200.0   1068509  Charged Off        B1


In [6]:
ddict = df.to_dict()
print(ddict['loan_amnt'][7])
ddict['sub_grade'][7]
g = zip(ddict['loan_amnt'].values(),ddict['sub_grade'].values(),ddict['id'].values(),ddict['loan_status'].values())
print(len(list(g)))

15000.0
45248


In [7]:
# create a training set
conn = sqlite3.connect(curr_path+"/lending-club-loan-data/database.sqlite")
df = pd.read_sql_query("select loan_amnt,id,loan_status,sub_grade from loan where loan_status !='Current' AND loan_status !='Late (31-120 days)' AND loan_status != 'Late (16-30 days)';", conn)
# print(df.axes)
# print(df.get_values()[0][1])
# print(df.get_values())
# print(df.values[3])
print(df.values[0])

conn.close()
# feature vector - predictor will predict ("charged off/default status") vs. 
# [1(maybe check this),'sub_grade','loan_amnt','emp_length','home_ownership','annual_inc','verification_status','dti'(debt-to-income),'']

[5000.0 '1077501' 'Fully Paid' 'B2']


In [8]:
# find all unique values for the loan status column
l_status_dict = defaultdict(int)
for row in df.itertuples():
    # print(row[3])
    if row[3] not in l_status_dict:
        l_status_dict[row[3]] += 1

# print(l_status_dict.keys())

In [9]:
print(l_status_dict.keys())

dict_keys(['Fully Paid', 'Charged Off', 'Default', 'In Grace Period', 'Does not meet the credit policy. Status:Fully Paid', 'Does not meet the credit policy. Status:Charged Off', 'Issued'])


In [10]:
conn = sqlite3.connect(curr_path+"/lending-club-loan-data/database.sqlite")
# very important, select a random sample of the data for our training set & validation sets 
df = pd.read_sql_query("select * from loan where id in(select id from loan order by random() limit 200000);", conn)
# df_validation = pd.read_sql_query("select * from loan where id in(selec id from loan order by random() limit 100000);", conn)


In [27]:
# split the data sets into training and validation
#for row in df.itertuples():
#   print(row[1])
# shuffle that dataframe
df = df.sample(frac=1).reset_index(drop=True)
data_train = df[:len(df)]
data_validation = df[len(df):]

In [28]:
#$ df.sum(1)
# print(df['loan_amnt'].sum(0))
#print(df['total_pymnt'].sum(0))
#print(df.shape)
display(data_train)

,index,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,94713,4767934,6030335.0,5450.0,5450.0,5450.000000,36 months,10.16%,176.27,B,...,NaN,NaN,NaN,NaN,NaN,NaN,43100.0,NaN,NaN,NaN
1,66710,28574268,31097476.0,35000.0,35000.0,35000.000000,36 months,16.99%,1247.68,D,...,NaN,NaN,NaN,NaN,NaN,NaN,76900.0,NaN,NaN,NaN
2,127327,20158922,22421593.0,13200.0,13200.0,13200.000000,60 months,14.99%,313.96,C,...,NaN,NaN,NaN,NaN,NaN,NaN,61000.0,NaN,NaN,NaN
3,154943,16361852,18464342.0,13000.0,13000.0,13000.000000,36 months,11.67%,429.74,B,...,NaN,NaN,NaN,NaN,NaN,NaN,24900.0,NaN,NaN,NaN
4,249047,51326801,54736527.0,9000.0,9000.0,9000.000000,36 months,6.89%,277.45,A,...,NaN,NaN,NaN,NaN,NaN,NaN,51300.0,NaN,NaN,NaN
5,409827,38484793,41278542.0,12000.0,12000.0,12000.000000,60 months,10.49%,257.87,B,...,NaN,NaN,NaN,NaN,NaN,NaN,40900.0,NaN,NaN,NaN
6,43629,65837053,70469869.0,12000.0,12000.0,12000.000000,36 months,13.67%,408.22,C,...,NaN,NaN,NaN,NaN,NaN,NaN,30800.0,NaN,NaN,NaN
7,258144,51147562,54557336.0,11000.0,11000.0,11000.000000,36 months,5.32%,331.27,A,...,NaN,NaN,NaN,NaN,NaN,NaN,125600.0,NaN,NaN,NaN
8,174195,1283671,1527266.0,10000.0,10000.0,10000.000000,36 months,13.67%,340.18,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,80356,63160376,67502254.0,10000.0,10000.0,10000.000000,36 months,18.55%,364.29,E,...,NaN,NaN,NaN,NaN,NaN,NaN,3700.0,NaN,NaN,NaN


In [ ]:
# need to build my feature vecotr
# X_train = f['sub_grade','loan_amnt']
# need y predictions combine(charged off/default(True(1)) or fully paid(False(0)))
# y_train =[]
def feature(data):
    feat = []
    # create our simplistic featur vector
    